<a href="https://colab.research.google.com/github/sedol1339/voice_score/blob/main/utils2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
URL = 'https://storage.googleapis.com/oleg-zyablov/misc/VoiceMOS'
!wget -q {URL}/main.zip
!wget -q {URL}/ood.zip
!wget -q {URL}/data_with_annotators.csv
!wget -q {URL}/data.csv
!unzip -q -o main.zip && rm main.zip
!unzip -q -o ood.zip && rm ood.zip

In [24]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from tqdm.notebook import tqdm

data = pd.read_csv('data.csv')
data.sample(3)

,subset,system,utterance,file,file_exists,score_mean,score_std,n_scores
3664,train,sys9d400,utt34664a2,sys9d400-utt34664a2.wav,True,4.000,0.9258,8
5767,train,sysff05c,utt4fd1f30,sysff05c-utt4fd1f30.wav,True,2.625,0.9161,8
5401,train,sysec937,uttb28aa53,sysec937-uttb28aa53.wav,True,2.375,0.9161,8


In [25]:
from sklearn.model_selection import KFold

train_and_val = data[data.subset.isin(['train', 'val'])]
kf = KFold(n_splits=5, shuffle=True, random_state=0)
data['fold'] = None
for i, (train, val) in enumerate(kf.split(train_and_val)):
  indices = train_and_val.index[val]
  data.loc[indices, 'fold'] = i

In [ ]:
import torch, torchaudio
import os, base64
from IPython import display

def get_waveform(wav_file):
  waveform, sample_rate = torchaudio.load(f'wav/{wav_file}')
  assert sample_rate == 16000
  return waveform[0]

def get_spectrogram(waveform, **kwargs):
  return torchaudio.transforms.Spectrogram(**kwargs)(waveform)

def show_audio(file_path, width=300):
  audio = open(file_path, 'rb').read()
  data_url = "data:audio/mp3;base64," + base64.b64encode(audio).decode()
  style = '''<style>audio::-webkit-media-controls-current-time-display,
    audio::-webkit-media-controls-time-remaining-display {display: none;}</style>'''
  display.display(display.HTML(style + f'<audio controls style="width: {width}px; ">'
                                       f'<source src="{data_url}"></audio>'))

def visualize_wav(waveform_or_filename):
  if type(waveform_or_filename) == str:
    waveform = get_waveform(waveform_or_filename)
    #display.display(display.Audio(f'wav/{waveform_or_filename}'))
    show_audio(f'wav/{waveform_or_filename}', width=760)
  else:
    waveform = waveform_or_filename
  fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(8, 6), dpi=100,
                                 gridspec_kw={'height_ratios': [1, 3]})
  ax1.plot(np.arange(len(waveform))[::20], waveform[::20])
  ax1.set_xlim(0, len(waveform))
  ax2.imshow(np.log(1e-6 + get_spectrogram(waveform)), aspect='auto')
  plt.show()

def show_random_waveform_with_score(dataframe, score_range=None, score_std_range=None):
  dataframe = dataframe[dataframe.file_exists]
  if score_range is not None:
    dataframe = dataframe[(dataframe.score_mean >= score_range[0]) & (dataframe.score_mean <= score_range[1])]
  if score_std_range is not None:
    dataframe = dataframe[(dataframe.score_std >= score_std_range[0]) & (dataframe.score_std <= score_std_range[1])]
  if len(dataframe) == 0:
    print('No samples')
    return
  row = dataframe.sample(1)
  row.drop(columns=['subset', 'system', 'utterance', 'file_exists'], inplace=True)
  display.display(row)
  visualize_wav(row.file.tolist()[0])
  
def get_wave2vec2_model(device='cpu'):
  bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
  return bundle.get_model().to(device)

def get_wave2vec2_output(model, waveforms, lengths=None, output_layers=['aux']):
  outputs = []
  x = waveforms
  transformer = model.encoder.transformer
  if x.ndim != 2:
    raise ValueError("Expected the input Tensor to be 2D (batch, time), but received {list(x.shape)}")
  #feature_extractor
  x = x[:, None, :]  # (batch, channel==1, frame)
  for i, layer in enumerate(model.feature_extractor.conv_layers):
      x, lengths = layer(x, lengths)  # (batch, feature, frame)
      if f'feature_extractor.conv_layers.{i}' in output_layers: outputs.append(x.transpose(1, 2))
  x = x.transpose(1, 2)  # (batch, frame, feature)
  if 'feature_extractor' in output_layers: outputs.append(x)
  #encoder
  x, mask = model.encoder._preprocess(x, lengths)
  if 'feature_projection' in output_layers: outputs.append(x)
  x = x + transformer.pos_conv_embed(x)
  x = transformer.layer_norm(x)
  x = transformer.dropout(x)
  if 'transformer.preprocess' in output_layers: outputs.append(x)
  for i, layer in enumerate(transformer.layers):
      if not (transformer.training and torch.rand(1).item() <= transformer.layer_drop):
          x = layer(x, attention_mask=None)
      if f'transformer.layers.{i}' in output_layers: outputs.append(x)
  if 'transformer' in output_layers: outputs.append(x)
  #aux
  x = model.aux(x)
  if 'aux' in output_layers: outputs.append(x)
  return outputs, lengths
  #raise ValueError(f"Unknown output_layer {output_layer}")

def get_wave2vec2_features_for_all_files(model, data, device, output_layer='aux', numpy=True):
  results = {}
  for i, row in tqdm(data.iterrows()):
    if row.file_exists:
      waveform = get_waveform(row.file)
      with torch.no_grad():
        outputs, lengths = get_wave2vec2_output(model, torch.Tensor(waveform[None]).to(device),
                                                output_layers=[output_layer])
      layer_output = outputs[0]
      layer_output_no_batch = layer_output[0]
      results[row.file] = layer_output_no_batch.cpu().detach()
  if numpy:
    results = {x: y.numpy() for x, y in results.items()}
  return results

```
Cлой feature_extractor.conv_layers.0: частота дискретизации 3199.8 гц, данные займут объем 195.776 Гб
Cлой feature_extractor.conv_layers.1: частота дискретизации 1599.8 гц, данные займут объем 97.880 Гб
Cлой feature_extractor.conv_layers.2: частота дискретизации 799.8 гц, данные займут объем 48.936 Гб
Cлой feature_extractor.conv_layers.3: частота дискретизации 399.8 гц, данные займут объем 24.464 Гб
Cлой feature_extractor.conv_layers.4: частота дискретизации 199.8 гц, данные займут объем 12.224 Гб
Cлой feature_extractor.conv_layers.5: частота дискретизации 99.9 гц, данные займут объем 6.112 Гб
Cлой feature_extractor.conv_layers.6: частота дискретизации 49.9 гц, данные займут объем 3.056 Гб
Cлой feature_projection: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.preprocess: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.0: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.1: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.2: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.3: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.4: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.5: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.6: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.7: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.8: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.9: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой transformer.layers.10: частота дискретизации 49.9 гц, данные займут объем 4.584 Гб
Cлой aux: частота дискретизации 49.9 гц, данные займут объем 0.191 Гб
```


In [ ]:
# device = 'cuda'
# wave2vec2_model = get_wave2vec2_model(device).eval()

# features_with_time_axis = get_wave2vec2_features_for_all_files(wave2vec2_model, data, device='cuda',
#                                           output_layer='feature_extractor.conv_layers.6')

In [ ]:
# Вариант 1: усреднение

# features = {file: x.mean(axis=0)
#             for file, x in features_with_time_axis.items()}

In [ ]:
# Вариант 2: среднее и среднекваратичное отклонение

# features = {file: np.concatenate([x.mean(axis=0), x.std(axis=0)])
#             for file, x in features_with_time_axis.items()}

In [ ]:
# Вариант 3: LSTM

# ...